In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import joblib
import re
from nltk.corpus import stopwords
import nltk

stop_words = stopwords.words('english')

def clr_stop_words(words):  ##clear stop words
    filter_words = []
    for w in words:
        if w not in stop_words:
            filter_words.append(w)

    return filter_words 

def save_model(model, fileName):
    # Save Model to file in the current working directory
    joblib.dump(model, fileName)
    
def load_model(fileName):    
    # Load from file
    joblib_model = joblib.load(fileName)
    return joblib_model

def cleanDescriptions(text):
    text = text.lower()
    text = re.sub('\n', ' ',text)# clean extra line
    text = re.sub('terms and condition.*$', '', text) 
    #text = re.sub('terms and condition*', '', text)
    #m = re.search('terms and condition.*', text)
    #if m:
    #    text = m.group(0)
    #re.match('terms and condition', text).group(0)
    return text

def clean_data(data):
    msg = data.lower() #lower all charracter
    #msg = deEmojify(lower_msg) ##clear emoji
    msg = re.sub('[^a-zA-Z0-9.]+', '  ', msg) ##clean symbol
    msg = re.sub('(.x?)http.*?(.*?)', ' ', msg) #clean url
    word_token = nltk.tokenize.word_tokenize(msg)
    #msg = spell_checker(word_token)
    word_token = clr_stop_words(word_token)
    #lemmatized = lemma_words(clean_stop_words)
    text = ' '.join(map(str, word_token))
    return text


In [21]:
model = load_model("svm_model.pkl")
vectorizer_ngram = load_model("vectorizer_ngram3.pkl")

In [8]:
englishDFClean = pd.read_csv("Result2.csv")

In [4]:
englishDFClean

,Name,Product Specifications,Product Description,Cleaned_Product_Specifications,Cleaned_Product_Description,Cleaned_Spec+Desc,Cleaned_Name,Cleaned_Only_Name,digit_uom,Cleaned_Only_Specifications,Cluster
0,AC TO DC ADAPTER 12V1A/12V2A/12V3A/9V1A/\nOV2A...,Category Shopee > Computer & Accessories > Com...,Specifications:\n\n+ Adapter Power Supply\n\n+...,chargers power supply,specifications adapter power supply input ac 1...,chargers power supply specifications adapter p...,ac dc adapter 12v1a 12v2a 12v3a 9v1a ov2a sv2a...,AC TO DC ADAPTER UK SWITCHING POWER SUPPL ... ...,"['12V1A/12V2A/12V3A/9V1A/', 'OV2A/SV2A/SV3A/',...",chargers power supply,9
1,[Ready Stock] Cooler Master High Performance T...,Category Shopee > Computer & Accessories > Com...,High Performance Thermal Paste (HTK-002-U1)\n\...,thermal paste brand cooler master,high performance thermal paste htk 002 u1 cool...,thermal paste brand cooler master high perform...,cooler master high performance thermal paste h...,[ Ready Stock ] Cooler Master High Performance...,['HTK-002-U1'],thermal paste,4
2,Gaz] (NEW) LAPTOP RAM PUSKILL 4GB 8GB DDR3 DDR...,Category Shopee > Computer & Accessories > Com...,IMPORTANT : PLEASE PM US YOUR LAPTOP MODEL BEF...,ram,important please pm us laptop model ordering c...,ram important please pm us laptop model orderi...,laptop ram puskill 4gb 8gb ddr3 ddr3l ddr4 pc3...,Gaz ] ( NEW ) LAPTOP RAM PUSKILL Low voltage S...,"['4GB', '8GB', 'DDR3', 'DDR3L', 'DDR4', 'PC3L'...",ram,12
3,RAM DIMM/SCDIMM 8GB/4GB/2GB DDR3/DDR3L/DDR2 DD...,Category Shopee > Computer & Accessories > Com...,* 100% Tested and Working\n\n* Refurbished\n\n...,ram,100 tested working refurbished brand mixed kin...,ram 100 tested working refurbished brand mixed...,ram dimm scdimm 8gb 4gb 2gb ddr3 ddr3l ddr2 dd...,RAM DIMM/SCDIMM PC Desktop Notebook Laptop Mem...,"['8GB/4GB/2GB', 'DDR3/DDR3L/DDR2', 'DDR4', '24...",ram,12
4,MB102 Solderless Mini Medium Large Breadboard ...,Category ‘Shopee > Computer & Accessories > Co...,Specification\n\n170 Holes (Small Size) = 3.5x...,chargers power supply,specification 170 holes small size 3.5x4.5cm 4...,chargers power supply specification 170 holes ...,mb102 solderless mini medium large breadboard ...,Solderless Mini Medium Large Breadboard Holes ...,"['MB102', '170', '400', '830']",chargers power supply,1
...,...,...,...,...,...,...,...,...,...,...,...
595,[ NEW ] PCIE Riser Card 009s 1X To 16X 6Pin US...,Category Shopee > Computer & Accessories > Com...,Features:\n\n1. Brand new and high quality 009...,graphic cards,features 1. brand new high quality 009s gold v...,graphic cards features 1. brand new high quali...,pcie riser card 009s 1x 16x 6pin usb3.0 gold l...,[ NEW ] PCIE Riser Card To Gold LED [ MSIA REA...,"['009s', '1X', '16X', '6Pin', 'USB3.0']",graphic cards,11
596,Male Female 2.1X5.5Mm De Power Plug Jack Adapt...,Category ‘Shopee > Computer & Accessories > Co...,Male / Female DC Connector 5.5*2.1mm Power Jac...,others,male female dc connector 5.5 2.1mm power jack ...,others male female dc connector 5.5 2.1mm powe...,male female 2.1x5.5mm de power plug jack adapt...,Male Female De Power Plug Jack Adapter Connect...,['2.1X5.5Mm'],others,9
597,[Lifetime]Spotify Gift Card Premium (Unlimited...,Category Shopee > Computer & Accessories > Com...,‘W Unlimited skips and shuffle\n\nNo adds\n\nP...,graphic cards,w unlimited skips shuffle adds premium feature...,graphic cards w unlimited skips shuffle adds p...,lifetime spotify gift card premium unlimited,[ Lifetime ] Spotify Gift Card Premium ( Unlim...,[],graphic cards,0
598,[PROMOTION] MCAFEE LIVESAFE 2021 ORIGINAL ANTI...,Category Shopee > Computer & Accessories > Com...,Attention!\nPlease read this before you buy Mc...,software brand mcafee software type security,attention please read buy mcafee product 11. m...,software brand mcafee software type security a...,mcafee livesafe 2021 original antivirus softwa...,[ PROMOTION ] MCAFEE LIVESAFE ORIGINAL ANTIVIR...,"['2021', '110']",software,8


In [26]:
import numpy as np


def prediction(text, model, vectorizer,csv_file):
    englishDFClean = pd.read_csv(csv_file)
    testing = [text]
    test_vec = vectorizer.transform(testing)
    #print(test_vec)
    y_pred = model.predict(test_vec)
    y_pred = y_pred[0]

    rslt_df = englishDFClean[englishDFClean['Cluster'] == y_pred] 
    corpus = rslt_df['Cleaned_Spec+Desc']

    corpus = corpus.tolist()
    corpus.append(text)

    vect = TfidfVectorizer(min_df=1, stop_words="english",ngram_range =(1,3))                                                                                                                                                                                                   
    tfidf = vect.fit_transform(corpus)                                                                                                                                                                                                                       
    pairwise_similarity = tfidf * tfidf.T 

    pairwise_similarity.toarray()  
                                                                                                                                                                                                                                  
    arr = pairwise_similarity.toarray()     
    np.fill_diagonal(arr, np.nan)                                                                                                                                                                                                                            
                                                                                                                                                                                                                 
    #input_doc = "chargers power supply specifications adapter power supply input ac 100 240v50 60hz cable length 90cm 140cm 300em output adaptor 5.5mm x 2.1mm 2.5mm . plug type uk 3 pin polarity inner positive outer negative led light indicator colour black features compact size high reliability light weight . professional dc112v1a 12v2a 12v3a 9v1a 9v2a 5v2a 6v2a switching power supply adapter high quality materials durable usage voltage protection load protection short circuit protection perfect power supply external hard drive wireless router cctv camera tv box ete . perfect power supply adapter led strip light stage lighting portable external hard drive wireless router cctv surveillance camera adsl modem mini tv mobile dvd tv box audio video equipments charging equipment mp3 mp4 small table lamp game console telephone lantern cameras surveillance equipment controllers home portable devices . please read purchasing note please make sure dc output connector size compatible device bidding . compatible charging electric toy car bike motor . indicator light change color accept returns exchanges unless item purchased defective . package include 1 x ac adapter cctv decoder dvbt2k2 adapter sv adapter 12v andriod adapterv"                                                                                                                                                                                                  
    input_idx = corpus.index(text)                                                                                                                                                                                                                      
    idx = (-arr[input_idx]).argsort()[:5]
    
    print("Similar Product")
    print("==============================================================================")
    for i in idx:
        print(rslt_df.iloc[i]['Cleaned_Name'], "\nConfidence Score: ", round(arr[input_idx][i],3))
        print("\n")
        
text = input('Enter the description:')
text = cleanDescriptions(text)
text = clean_data(text)
prediction(text, model, vectorizer_ngram, "Result2.csv")


Enter the description:adata ram ddr4
Similar Product
adata ram ddr4 ddr4 2400 2666 u dimm sodimm 4gb 8gb 16gb 32gb laptop ddr3 2gb 4gb 8gb 
Confidence Score:  0.109


adata ram 4gb 8gb ddr4 2666mhz ram sodimm lodimm memory notebook laptop desktop pc kingston ram 
Confidence Score:  0.104


ram adata 4gb 8gb 16gb ddr4 2400mhz 2666mhz 3200mhz sodimm notebook laptop 
Confidence Score:  0.089


adata sodimm ddr4 ram 2666mhz 4gb 8gb 16 gb laptop 
Confidence Score:  0.087


adata ram ddr4 ddr4 2400 2666 u dimm dimm 4agb 8gb 16gb 32gb 2666 mhz 
Confidence Score:  0.075


